In [1]:
%run OrderBookPlots.ipynb

part = []

whole = ['a', 'b', 'c' ,'d']

print(len(whole))

whole_ser = pd.Series(whole)
whole_ser[~whole_ser.isin(part)].to_list()

In [2]:
# the algorithm (CC.java) is taken from https://algs4.cs.princeton.edu/41graph/

class OrderBook2OrderBookPlots:
    def __init__(self, taq_df,
              row_id_col, # unique row id
              sec_col, # column with security code
              link_col, # orderno or some mark linking trades and quotes
              trdno_col,
              only_secs = []
             ):
        self.OrderBookPlots_df = taq_df.copy()
        self.row_id_col = row_id_col
        self.sec_col = sec_col
        self.link_col = link_col
        self.trdno_col = trdno_col
        
        self.secs = pd.unique(taq_df[sec_col])
        if len(only_secs) > 0:
            only_secs_ser = pd.Series(only_secs)
            self.wrong_secs = only_secs_ser[~only_secs_ser.isin(self.secs)].to_list()
            if len(self.wrong_secs) > 0:
                self.OrderBookPlots_df =pd.DataFrame()
                raise SystemExit(f"Wrong SECCODES: {self.wrong_secs}")
            else:
                self.secs = only_secs
        self.OrderBookPlots_df = self.OrderBookPlots_df[self.OrderBookPlots_df.SECCODE.isin(self.secs)]
        
        self.vertex_count_dic = {k:v for (k, v) in zip(self.secs, [0 for i in range(len(self.secs))])}
        self.rowid2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.lnk2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.trdno2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.edge_tpls = {k:v for (k, v) in zip(self.secs, [[] for i in range(len(self.secs))])}
        
        self.rows_to_edges()
        
        self.Gs = {k:v for (k, v) in zip(self.secs, [Graph(self.edge_tpls[sec],
                                                           self.vertex_count_dic[sec]) for sec in self.secs])}
        self.OBPs = {k:v for (k, v) in zip(self.secs, [OrderBookPlots(self.Gs[sec]) for sec in self.secs])}
        
        self.to_OBPlots_df()

    def sec_rowid_lnk2edges(self, rowid, sec, lnk, trdno): 
        # give 0, 1, 2, ... vertex names to all raw ids and "links" (eg ordernos)
        if rowid not in self.rowid2gkey_dic[sec]:
            self.rowid2gkey_dic[sec][rowid] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        if lnk not in self.lnk2gkey_dic[sec]:
            self.lnk2gkey_dic[sec][lnk] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        if trdno not in self.trdno2gkey_dic[sec]:
            self.trdno2gkey_dic[sec][trdno] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        self.edge_tpls[sec].append((self.rowid2gkey_dic[sec][rowid], 
                               self.lnk2gkey_dic[sec][lnk]))
        if pd.notna(trdno):
            self.edge_tpls[sec].append((self.rowid2gkey_dic[sec][rowid], 
                               self.trdno2gkey_dic[sec][trdno]))

    def rows_to_edges(self):
        [self.sec_rowid_lnk2edges(rowid, sec, lnk, trdno) 
         for rowid, sec, lnk, trdno in zip(self.OrderBookPlots_df[self.row_id_col],
                                    self.OrderBookPlots_df[self.sec_col],
                                    self.OrderBookPlots_df[self.link_col],
                                    self.OrderBookPlots_df[self.trdno_col])]
        
    def to_OBPlots_df(self):
        self.OrderBookPlots_df['OBPLOTNO'] = [self.OBPs[sec].OBPlotnos[self.rowid2gkey_dic[sec][rowid]] 
                                    for (sec, rowid) in zip(self.OrderBookPlots_df['SECCODE'], 
                                                            self.OrderBookPlots_df['NO'])]

In [3]:
OrderLog = 'OrderLog20020812'
#OrderLog = 'OrderLog20071001'
OrderLog_df = pd.read_csv('./resources/' + OrderLog + '.txt')

OrderLog_df = OrderLog_df.astype({"NO":"int",
                                  "SECCODE":"string",
                                  "BUYSELL":"string",
                                  "TIME":"string",
                                  "ORDERNO":"int",
                                  "ACTION":"int",
                                  "PRICE":"float",
                                  "VOLUME":"int",
                                  "TRADENO":pd.Int64Dtype(),
                                  "TRADEPRICE":"float"
                                 })
OrderLog_df

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,1,RU14VOST1005,S,103000000,1,1,2.380,4500,<NA>,NaN
1,2,RU0009071187,S,103000000,2,1,990.000,300,<NA>,NaN
2,3,RU14VOST1005,S,103000000,3,1,2.420,21600,<NA>,NaN
3,4,RU0009100762,B,103000000,4,1,0.133,100000,<NA>,NaN
4,5,RU0009054449,S,103000000,5,1,332.000,500,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...
90739,90740,RU14TATN3006,B,184557000,38574,1,18.770,50000,<NA>,NaN
90740,90741,RU14VOST1005,S,184558000,38559,0,1.990,49800,<NA>,NaN
90741,90742,RU0009046700,S,184559000,38575,1,18.450,100000,<NA>,NaN
90742,90743,RU0009071187,B,184559000,38550,0,725.840,1,<NA>,NaN


In [4]:
OrderBookPlots_df = OrderBook2OrderBookPlots(OrderLog_df, 'NO', 'SECCODE', 'ORDERNO', 'TRADENO', []).OrderBookPlots_df
OrderBookPlots_df.to_csv('./resources/' + OrderLog + '.csv')
OrderBookPlots_df

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,OBPLOTNO
0,1,RU14VOST1005,S,103000000,1,1,2.380,4500,<NA>,NaN,0
1,2,RU0009071187,S,103000000,2,1,990.000,300,<NA>,NaN,0
2,3,RU14VOST1005,S,103000000,3,1,2.420,21600,<NA>,NaN,2
3,4,RU0009100762,B,103000000,4,1,0.133,100000,<NA>,NaN,0
4,5,RU0009054449,S,103000000,5,1,332.000,500,<NA>,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
90739,90740,RU14TATN3006,B,184557000,38574,1,18.770,50000,<NA>,NaN,209
90740,90741,RU14VOST1005,S,184558000,38559,0,1.990,49800,<NA>,NaN,161
90741,90742,RU0009046700,S,184559000,38575,1,18.450,100000,<NA>,NaN,357
90742,90743,RU0009071187,B,184559000,38550,0,725.840,1,<NA>,NaN,230


In [5]:
OrderBookPlots_df[(OrderBookPlots_df.SECCODE == 'RU0009024277') & 
                 (OrderBookPlots_df.OBPLOTNO == 30)]

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,OBPLOTNO
480,481,RU0009024277,B,103000000,481,1,480.00,20,<NA>,NaN,30
1122,1123,RU0009024277,B,103000000,1123,1,480.02,200,<NA>,NaN,30
2155,2156,RU0009024277,B,103135000,1866,1,480.01,500,<NA>,NaN,30
3866,3867,RU0009024277,B,103701000,2834,1,480.01,50,<NA>,NaN,30
3880,3881,RU0009024277,B,103705000,2847,1,480.01,1,<NA>,NaN,30
...,...,...,...,...,...,...,...,...,...,...,...
13656,13657,RU0009024277,S,111707000,7550,2,480.00,138,29846687,480.0,30
13799,13800,RU0009024277,S,111737000,7624,1,480.00,100,<NA>,NaN,30
13801,13802,RU0009024277,B,111737000,2937,2,480.00,100,29846711,480.0,30
13802,13803,RU0009024277,S,111737000,7624,2,480.00,100,29846711,480.0,30


In [6]:
# OrderBookPlots_df.to_csv('./resources/OrderLog20020812_OBP.csv')